In [1]:
import pandas as pd; import numpy as np; import os as os; import multiprocessing as mp
import matplotlib.pyplot as plt; import seaborn as sns;  import scipy.interpolate as sp
import statsmodels.formula.api as smf
%matplotlib inline

In [2]:
if os.name=="nt":
    path ="J:\\Project\\Cost_Effectiveness\\dev\\"
else:
    path = "\home\\j\\Project\Cost_Effectiveness\\dev\\"
    
ltable_path ="data\\gbd"
all_data = "data_processed"

### Load in Data

In [3]:
all_death_df = pd.read_csv(path+'\\data_processed\\Mortality_Rates.csv')
ihd_death_df = pd.read_csv(path +'\\data_processed\\ihd_mortality_rate.csv')
ihd_incidence_df = pd.read_csv(path + '\\data_processed\\IHD incidence rates.csv')
cohort = pd.read_csv(path+all_data+"\\level2_population.csv")
cohort.rename(columns={'age':'Age', 'year_id':'Year', 'IHD':'IHD incidence'}, inplace=True) ## this needs to change
ltable = pd.read_csv(path+ltable_path+ "\\interpolated_reference_life_table.csv")

## Define an interpolator


In [4]:
### interpolator
## Per Alec's sugeestion turn this into a class
def gen_curve(df, df_id='Mortality_Rate', time =24, inv=0):
    df['inv_prob'] =  np.exp(-df[df_id])
    df.Year -= np.min(df.Year)
    df.Age -= df.Year 
    df = df[df.Age>0]
    df = df[df.Age<=80]
    df['cumprod'] = df.groupby(['Age', 'sex']).inv_prob.agg(np.cumprod)
    
    df.Year+=1
    # could not control when random num was out of bounds high or low
    ## function defined below takes twice the time now... not worrying about that
    for i in np.arange(2):
        for s in np.unique(df.sex):
                temp = pd.DataFrame(columns=df.columns)
                temp['Age'] =np.unique(df.Age)
                temp['sex']= s
                temp['Year']=np.where(i==1, 0, time)
                temp['cumprod'] = i
                df = df.append(temp)
            
    model= sp.LinearNDInterpolator(df[['cumprod','Age', 'sex']], df['Year'])
    ## I need to backcalc time didn't know of a better way.
    ## I could add another dim to interpolation (current year(i.e. self.clock),
    #but python stalled out in calcing that many dims
    inv_model = sp.LinearNDInterpolator(df[['Year','Age', 'sex']],df['cumprod'])
    
    def func(self, ind, rand_seq, name):
               
        df =self.attributes.iloc[ind]
        # convert into rate ## check if there is an easier way or if the math works. do I need to worry about time?
        rand_seq = -np.log(1-(1-rand_seq))
        rand_seq += self.rate_adj[name][ind,0].reshape(len(ind[0],))## check the ordering
        rand_seq*= self.rate_adj[name][ind,1].reshape(len(ind[0],))
        rand_seq = np.exp(-rand_seq)
        df['cumprod'] =rand_seq.T
        fit =model(df[['cumprod','Age', 'sex']])
        fit = np.round(fit,3)
        return fit
    
    def invfunc(self,ind, name):
        df =self.attributes.iloc[ind]
        time = self.clock[ind]
        df['Year'] =time
        fit =np.array(inv_model(df[['Year','Age', 'sex']]))
        # convert into rate ##  check if there is an easier way or if the math works
        fit = -np.log(1-(1-fit))
        fit /= self.rate_adj[name][ind,1].reshape(len(ind[0]),)
        fit -= self.rate_adj[name][ind,0].reshape(len(ind[0]),)
        fit = np.exp(-fit)
        fit = np.round(fit,3)
        return fit
    
           
    return {'func':func, 'invfunc': invfunc}

### Define sim

In [43]:
class sim(object):
    '''TODO: add documentation, write some test to avoid duplicates
    FIX: IHD dead indicator'''
    

    def __init__(self,n=2e4, time =24):
        
        self.n = int(n)
        self.time = int(time)
        self.init_attributes = pd.DataFrame()
        
        
        ## define stuff that will be used later on
        self.events= {}
        self.event_names =[]
        self.updates = {}
        self.deadly_events = []
        self.init_rate_adj = {}
        self.init_at_risk = pd.DataFrame()
     
    def add_attribute(self,colnames, data):
        ## should write in a check to see if att is already defined.
        self.init_attributes[colnames]=data
    def add_events(self,name, funcs_dict, deadly, at_risk_ind ):
        self.event_names.append(name)
        self.events.update({name: funcs_dict})
        if deadly==1:
            self.deadly_events.append(name)
        self.init_rate_adj.update({name: np.vstack((np.zeros(self.n) ,np.ones(self.n))).T})
        self.init_at_risk[name] = at_risk_ind
        
    def add_updates(self,name, funcs_dict):
        self.updates.update({name:funcs_dict})
    
    
    def run(self, seed=1):
        np.random.seed(1)
        ## so I can run multiple times... is there a better way?
        self.attributes = self.init_attributes.copy() ## attributes just stores sex, age in 1990, dichotomous events 
        self.rate_adj =self.init_rate_adj.copy()
        self.at_risk = self.init_at_risk.copy()
        self.ledger = pd.DataFrame(columns=['simulant_id', 'event_name', 'event_time']) ## records events and time they occur
        self.clock = np.repeat(0, self.n)
      
        
        ## get baseline events
        for events in self.event_names:
            self.ledger =self.ledger.append(pd.DataFrame({ 'simulant_id': \
                                 self.attributes[self.attributes[events]==1]['simulant_id'],
                                'event_name': events,
                                'event_time': 0}))
        
        #set indicator to index attributes
        ind = np.where(self.attributes.Dead==0)
        
        ## run till I kill everyone. 
        while np.sum(self.attributes.Dead==0):
            temp_log = np.empty([len(ind[0]), len(self.events)])
            rand = np.random.uniform(0,1, [len(ind[0]), len(self.events)])
            
            ## loop thru events ## probably should add in 
            for i in np.arange(len(self.events)):
                name = self.event_names[i]
                ## need to find out where time t occurs on the [0,1] range and scale new random numbers
                ## this is by far the slowest step and there are other ways this can be done 
                time_scale = self.events[name]['invfunc'](self, ind, name )*rand[:,i]
              
                ## get time of event
                temp_log[:,i] = self.at_risk[name].iloc[ind]*999+self.events[name]['func'](self, ind, time_scale, name)
            
            
           
            ## get details on soonest event
            min_ind =temp_log.argmin(axis=1) # get templog index
            min_time = temp_log.min(axis=1) # get time it occured
            min_name = np.array([self.event_names[i] for i in min_ind]) ## get min event name
            

            ## update ledger
            self.ledger = self.ledger.append(pd.DataFrame({'simulant_id':self.attributes.iloc[ind]['simulant_id'].values,
                            'event_name': min_name,
                            'event_time':min_time}))
            # update run time
            self.clock[ind] = min_time.copy()
            ## update attributes based on events that occured
            for i in np.arange(len(self.updates)):
                name = self.updates.keys()[i]
                self.updates[name](self, ind, min_ind, min_name)
            
            ## kill people off
            self.attributes['Dead'].iloc[ind] =np.where(np.in1d(min_name, self.deadly_events), 1, 0)
            ind =np.where(self.attributes.Dead==0)
           
        

In [44]:
###Not the final solution. Not even sure where this belongs
def daly_calc(self):
       
        ## may change this in the future
        self.ledger.event_name = np.where(self.ledger.event_name=='IHD Dead', 'Dead', self.ledger.event_name)
        
        ## calc ylds
        dis = self.ledger[self.ledger.event_name=='IHD incidence']
        dead = self.ledger[self.ledger.event_name=='Dead']
        dis = pd.merge(dis, dead, on='simulant_id', suffixes=('_dis', '_dead'))
        ylds = np.sum(dis.event_time_dead -dis.event_time_dis)*.2

        #calc ylls
        df = pd.merge(self.attributes, dead, on='simulant_id')
        df['death_age']= df.Age+df.event_time
        df['death_age']= np.round(df['death_age'].values)# because life table doesn't have decimals
        df = pd.merge(df, ltable, left_on='Age', right_on= 'age')
        return df.ex.sum() +ylds
def cost_calc(self):
    df = self.ledger.copy()
    tx = df[df.event_name=='tx']
    dead = df[df.event_name=='Dead']
    tx = pd.merge(tx, dead, on='simulant_id', )
    return np.sum((tx.event_time_y - tx.event_time_x))*2
    

# Run Level 2 sim


In [45]:
###################
#add attributes
model = sim()
model.add_attribute(cohort.columns,cohort)
model.add_attribute('IHD Dead', 0) ## in case we want to seperate by death
model.add_attribute('tx', 0)


### Run w/o intervention

In [46]:
######################################
#add time-to-event events

## when all said and done... not sure this loop is worthwhile. There may be a better way.
event_col= ['Mortality_Rate', 'Mortality_Rate', 'Incidence']
event_data = [all_death_df, ihd_death_df, ihd_incidence_df]
event_names = ['Dead', 'IHD Dead', 'IHD incidence']
dead_events = [ 1, 1, 0]
ls_at_risk = [model.init_attributes.Dead, np.where(model.init_attributes['IHD incidence']==1, 0,1),  
              model.init_attributes['IHD incidence']]

for i in np.arange(len(event_col)):
      
    name = event_names[i]
    model.add_events(name, gen_curve(event_data[i], event_col[i]), dead_events[i],ls_at_risk[i])

#######################################
## add updates

# define updates
def ihd_update(self, ind, min_ind, min_name):
    self.attributes['IHD incidence'].iloc[ind]= np.where(min_name=='IHD incidence'  , 1, self.attributes['IHD incidence'].iloc[ind])
    
    self.at_risk['IHD Dead'].iloc[ind] = np.where(min_name=='IHD incidence' , 0,self.at_risk['IHD Dead'].iloc[ind] )
    self.at_risk['IHD incidence'].iloc[ind] = np.where(min_name=='IHD incidence', 1, self.at_risk['IHD incidence'].iloc[ind])

model.add_updates('ihd_update', ihd_update)

In [ ]:
## run the model.
collect_notx = np.empty([100,2])
for i in np.arange(100):
    model.run(seed=i)
    collect_notx[i,:] = np.array([i,daly_calc(model)])


C:\Users\mwm6\AppData\Local\Continuum\Miniconda2\lib\site-packages\ipykernel\__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mwm6\AppData\Local\Continuum\Miniconda2\lib\site-packages\ipykernel\__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


## Run w/ intervention
#### add tx updates

In [29]:
##################################
# add tx

#define tx event
def txfunc(self, ind, rand_seq, name):
    df = self.attributes.iloc[ind]
    ## over 25 by the time 1995 rolls around get tx.
    tx_ind = np.where((df.Age>=20) & (df.tx==0) & (5>= self.clock[ind]), 5, 999) 
    ## for those under 25 by the time 25 rolls around 
    tx_ind = np.where((df.tx==0) & (df.Age<20) & (25>=df.Age+self.clock[ind]), 25-df.Age, tx_ind)
    return tx_ind
def txinvfunc(self,ind, name ):
    return 1
## add tx
model.add_events('tx', {'func':txfunc, 'invfunc': txinvfunc},0,0)


#######################################
## add updates


def tx_update(self, ind, min_ind, min_name):
    
    self.attributes['tx'].ix[ind] = np.where(min_name=='tx' , 1, self.attributes['tx'].ix[ind])
    self.at_risk['tx'].ix[ind] = np.where(min_name=='tx', 1, self.at_risk['tx'].ix[ind])
    self.rate_adj['IHD incidence'][ind,1] *=np.where(min_name=='tx', 2, 1)
    
    
model.add_updates('tx_update', tx_update)

In [37]:
## run the model.
collect_tx = np.empty([100,3])
for i in np.arange(100):
    model.run(seed=i)
    collect_tx[i] = np.array([i,daly_calc(model), cost_calc(model)])


C:\Users\mwm6\AppData\Local\Continuum\Miniconda2\lib\site-packages\ipykernel\__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\mwm6\AppData\Local\Continuum\Miniconda2\lib\site-packages\ipykernel\__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
